In [1]:
import pandas as pd

import numpy as np

from nltk.corpus import stopwords

import pysentiment2 as ps

import nltk

import pysentiment2 as ps

from nltk.stem.porter import *
import os

In [2]:
def countWords(documentTokens, listNegativeWords):
    '''
    This functions will return a dictionary with keys being the masterDictionary keys and values are the number of times
    the word is in the master dictionary
    Parameters:
        documentTokens: (list) tokens of the document
        listNegativeWords: (list) negative words
    '''
    #listNegativeWords = set(listNegativeWords)
    #for token in documentTokens:
    #    if token in listNegativeWords:
    
            
    masterDictionary = dict(zip(listNegativeWords, [0]*len(listNegativeWords)))
    for word in documentTokens:
        if word in masterDictionary.keys():
            masterDictionary[word] += 1
            
    return masterDictionary

In [3]:
#Load the Returns file
df = pd.read_csv('../DataRetrieval/url_with_returns.csv')[['reportDate','filingDate', 'cik', 'ExcessReturnsEqualWeightedSnP', 'ExcessReturnsValueWeightedSnP']]
df['cik'] = df['cik'].astype(np.int).astype(str).str.pad(10, fillchar='0')
df = df[df.reportDate >= '2016-01-01']
print(df.shape)
df.head()

(11383, 5)


C:\Users\JuliJaramillo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\JuliJaramillo\AppData\Local\Temp\ipykernel_22120\2655062236.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df['cik'] = df['cik'].astype(np.int).astype(str).str.pad(10, fillchar='0')


,reportDate,filingDate,cik,ExcessReturnsEqualWeightedSnP,ExcessReturnsValueWeightedSnP
20,2016-03-31,2016-05-09,0000814453,0.015216,0.012411
21,2016-06-30,2016-08-09,0000814453,0.010468,0.012448
22,2016-09-30,2016-11-08,0000814453,-0.077190,-0.069763
23,2016-12-31,2017-03-01,0000814453,-0.000803,-0.003919
24,2017-03-31,2017-05-10,0000814453,0.001553,-0.002548


In [4]:
#Define the dictionaries

#HarvardDictionary = list(ps.HIV4()._negset)
#LMDictionary = list(ps.LM()._negset)

LMDictionary = pd.read_excel('LM_Neg.xlsx', sheet_name = 'ND_FinTerms_Negative_v2', header = None).values.reshape(-1)
LMDictionary = [w.lower() for w in LMDictionary ]
HarvardDictionary = list(pd.read_csv('Harvard_Neg.csv').columns)
HarvardDictionary = [w.strip().lower() for w in HarvardDictionary ]

#Define the stopwords dictionary
stop_words = list(set(stopwords.words('english')))

C:\Users\JuliJaramillo\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
proportion = []
listFileNames = os.listdir('../DataRetrieval/Forms/')

for row, fileName in enumerate(listFileNames):
    
    cik = fileName.split('_')[0]
    filingDate = fileName.split('_')[1].split('.')[0]

    
    path = '../DataRetrieval/Forms/{}_{}.txt'.format(cik, filingDate)
    
    f = open(path, "r")
    text = f.read()
    
    tokensOwn = nltk.word_tokenize(text)
    filteredText = [w for w in tokensOwn if not w in stop_words]
    
    #lemmatized = [PorterStemmer().stem(w) for w in filteredText]
    
    countingLM = countWords(filteredText, LMDictionary)
    countingHAR = countWords(filteredText, HarvardDictionary)
    
    propLM = sum(list(countingLM.values())) /  len(filteredText)
    propHar = sum(list(countingHAR.values())) /  len(filteredText)
    
    proportion.append( (cik, filingDate, propLM, propHar) )
    
    f.close()
    if (row % 100 == 0) & (row != 0): 
        print(row)
        break


100


In [6]:
newDf = pd.DataFrame(proportion, columns = ['cik', 'filingDate', 'LM_Prop', 'Har_Prop'])
newDf.head()

,cik,filingDate,LM_Prop,Har_Prop
0,0000001800,2016-05-04,0.021715,0.043840
1,0000001800,2016-08-03,0.023491,0.043971
2,0000001800,2016-11-03,0.025859,0.046570
3,0000001800,2017-02-17,0.024818,0.039822
4,0000001800,2017-05-03,0.020300,0.039862
